# Pesquisando por iniciativas/projetos com Dados Abertos Governamentais no Github

In [1]:
import requests
import pandas as pd
import time

TODO:

**Mapeamento das principais bibliotecas**

- [API da Câmara dos deputados](https://dadosabertos.camara.leg.br/swagger/api.html)

In [2]:
search_strings_libraries = []

Devido a estrutura de pesquisa no github as palavras chaves utilizadas para o 
news não foram tão eficiente, visto que preciso adicionar palavras de busca ou 
de forma conter uma das palavras ou conter todas as palavras na sequencia especificada.

Sendo assim:
- '"projeto" and "dados governo"',
- '"projeto" and "dados governamentais"',
- '"projeto" and "monitora" and "dados" and "governo"'

Não foram buscas efetivas, porém as palavras chaves inicias sim.

In [3]:
search_strings = [
            'dados abertos',
            'dados abertos brasil',
            'dados abertos governo',
            'dados abertos governamentais',
            'dados governamentais',
            'dados publicos abertos',
            'dados do governo',
            'analise de dados do governo',
            'analise de dados governamentais',
            'portal de dados do governo',
            'portal de dados governamentais',
            'portal publico do governo',
            'portal de dados abertos do governo',
        ]

'consumir' e 'publicação' não retornaram resultados e por isso foram retirados a fim de adicionar palavras que retornem resultados melhores
- 'consumir dados abertos do governo'
- 'consumir dados abertos governamentais'
- 'publicação de dados abertos do governo',
- 'publicação de dados governamentais',

In [4]:
sort = '&sort=stars&order=desc'
url_base = 'https://api.github.com/search/repositories?q='
credentials=('lorenaps','ba6db863961f1221ef38f7294976f0dc8dc36731')

Verificando o limite de requisições

In [5]:
t = requests.get('https://api.github.com/rate_limit', auth=credentials)
t.json()

{'rate': {'limit': 5000, 'remaining': 4912, 'reset': 1573177642},
 'resources': {'core': {'limit': 5000, 'remaining': 4912, 'reset': 1573177642},
  'graphql': {'limit': 5000, 'remaining': 5000, 'reset': 1573177733},
  'integration_manifest': {'limit': 5000,
   'remaining': 5000,
   'reset': 1573177733},
  'search': {'limit': 30, 'remaining': 30, 'reset': 1573174193}}}

In [69]:
columns=[
        'id',
        'full_name',
        'description',
        'owner_type', 
        'owner_api_url',
        'owner_url',
        'url',
        'api_url',
        'fork',
        'created_at',
        'updated_at',
        'size',
        'stargazers_count',
        'language',
        'has_issues',
        'has_wiki',
        'forks_count',
        'forks',
        'open_issues',
        'watchers',
        'timestamp_extract'
    ]


In [68]:
def add_resultado(item):
    
    df = pd.DataFrame([[
                        item.get('id'),
                        item.get('full_name', None),
                        item.get('description', None),      
                        item.get('owner').get('type', None),
                        item.get('owner').get('url', None),
                        item.get('owner').get('html_url', None),
                        item.get('html_url', None),
                        item.get('url', None),
                        item.get('fork', None),
                        item.get('created_at', None),
                        item.get('updated_at', None),
                        item.get('size', None),
                        item.get('stargazers_count', None),
                        item.get('language', None),
                        item.get('has_issues', None),
                        item.get('has_wiki', None),
                        item.get('forks_count', None),
                        item.get('forks', None),
                        item.get('open_issues', None),
                        item.get('watchers', None),
                        str(time.time()).split('.')[0]]], columns=columns)

    return df    

In [8]:
def extrair_dados(data, resultados):
    
    for item in data.get('items', None):
        
        resultados = pd.concat([resultados, add_resultado(item)], ignore_index=True, sort=False)

    return resultados

Verificando limitação de extração de dados da API

In [9]:
mais_de_mil = 'https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc&page=35'
t = requests.get(mais_de_mil, auth=credentials)
t.json()

{'documentation_url': 'https://developer.github.com/v3/search/',
 'message': 'Only the first 1000 search results are available'}

In [10]:
repositories_df = pd.DataFrame(columns=['id',
                                    'full_name',
                                    'description',
                                    'owner_type', 
                                    'owner_api_url',
                                    'owner_url',
                                    'url',
                                    'api_url',
                                    'fork',
                                    'created_at',
                                    'updated_at',
                                    'size',
                                    'stargazers_count',
                                    'language',
                                    'has_issues',
                                    'has_wiki',
                                    'forks_count',
                                    'forks',
                                    'open_issues',
                                    'watchers',
                                    'timestamp_extract',
                                    'commits',
                                    'contributors',]) 

In [11]:
def percorrendo_paginas(url, repositories_df):
    
    print('\nPrimeira requisição')
    
    results = requests.get(url, auth=credentials)    
    data = dict(results.json())
    total = data.get('total_count', None)
        
    print(">>> Foram encontrados {0} resultados. Extraindo...".format(total))

    repositories_df = extrair_dados(data, repositories_df)
    
    iteracoes = total // 30 
    
    for iteracao in range(0, iteracoes):        
        header = dict(results.links)
        
        if header.get('next', False):
            next_url = header.get('next').get('url')
            
            print("\nNext url: {0}".format(next_url))
            
            results = requests.get(next_url, auth=credentials)
            data = dict(results.json())
            repositories_df = extrair_dados(data, repositories_df)
        
    return resultados

In [12]:
for string in search_strings:
    %%time
    url = url_base + string + sort
    print("\nExtraindo repositórios para a string: '{0}'".format(string))
    resultados = percorrendo_paginas(url, resultados)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs

Extraindo repositórios para a string: 'dados abertos'

Primeira requisição
>>> Foram encontrados 370 resultados. Extraindo...

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=2

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=3

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=4

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=5

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=6

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=7

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=8

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=9

Next url: https:

In [13]:
resultados.describe()

,id,full_name,description,owner_type,owner_api_url,owner_url,url,api_url,fork,created_at,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,timestamp_extract,commits,contributors
count,570,570,537,570,570,570,570,570,570,570,...,422,570,570,570,570,570,570,570,0.0,0.0
unique,409,409,376,2,356,356,409,409,1,409,...,22,2,2,16,16,14,25,31,0.0,0.0
top,195294598,thenets/ckanext-dadosabertos,Plugin / Tema do Portal de Dados Abertos do Go...,User,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/thenets/ckanext-dadosabertos,https://api.github.com/repos/prodest/ckanext-d...,False,2019-07-04T19:52:41Z,...,Python,True,True,0,0,0,0,1573174215,NaN,NaN
freq,6,6,6,463,16,16,6,6,570,6,...,94,556,553,423,423,488,390,30,NaN,NaN


## Extraindo Commits e Contributors

In [14]:
def extract_commits(url_repo):
    
    commits_url = url_repo + '/commits'  
    results = requests.get(commits_url, auth=credentials)
    
    # No caso do repositório estar vazio
    if results.status_code is 409:
        return None
    
    commits = len(results.json())

    header = dict(results.links)
    
    while header.get('next', False):
        next_url = header.get('next').get('url')        
        results = requests.get(next_url, auth=credentials)
        commits = commits + len(results.json())    
        header = dict(results.links)


    return commits

In [15]:
def extract_contributors(url_repo):
    
    contributors_url = url_repo + '/contributors'
    results = requests.get(contributors_url, auth=credentials)
    
    # No caso do repositório estar vazio
    if results.status_code is 204:
        return None
    
    contributors = len(results.json())

    header = dict(results.links)
    
    while header.get('next', False):
        next_url = header.get('next').get('url')
        results = requests.get(next_url, auth=credentials)
        contributors = contributors + len(results.json())
        header = dict(results.links)
    
    return contributors

In [16]:
urls = resultados['api_url']

for url in urls:
        
    print('\n>>> ', url)
    
    repo = requests.get(url, auth=credentials)
    repo = dict(repo.json())
        
    commits = extract_commits(url)
    contributors = extract_contributors(url)

    print("Tem {0} Commits - {1} Contributors".format(commits,contributors))

    resultados.loc[resultados["api_url"] == url, 'commits'] = commits
    resultados.loc[resultados["api_url"] == url, 'contributors'] = contributors


>>>  https://api.github.com/repos/CamaraDosDeputados/dados-abertos
Tem 28 Commits - 4 Contributors

>>>  https://api.github.com/repos/dadosgovbr/catalogos-dados-brasil
Tem 42 Commits - 4 Contributors

>>>  https://api.github.com/repos/prefeiturasp/dados-educacao
Tem 18 Commits - 2 Contributors

>>>  https://api.github.com/repos/dadosgovbr/aplicativos-dados-brasil
Tem 40 Commits - 5 Contributors

>>>  https://api.github.com/repos/dadosgovbr/kit
Tem 361 Commits - 6 Contributors

>>>  https://api.github.com/repos/mapaslivres/localidades
Tem 69 Commits - 5 Contributors

>>>  https://api.github.com/repos/odufrn/odufrn-downloader
Tem 237 Commits - 6 Contributors

>>>  https://api.github.com/repos/brasilopen/brasilopen
Tem 10 Commits - 2 Contributors

>>>  https://api.github.com/repos/vitorbaptista/dados-abertos-camara.gov.br
Tem 31 Commits - 1 Contributors

>>>  https://api.github.com/repos/jonny-data/conheca-seu-vereador
Tem 70 Commits - 6 Contributors

>>>  https://api.github.com/repos/da

Tem 85 Commits - 2 Contributors

>>>  https://api.github.com/repos/vgeorge/estados-brasileiros
Tem 1 Commits - 1 Contributors

>>>  https://api.github.com/repos/paraibatransparente/dados
Tem 95 Commits - 2 Contributors

>>>  https://api.github.com/repos/CodeForCuritiba/Analises-Jupyter-Notebook
Tem 2 Commits - 1 Contributors

>>>  https://api.github.com/repos/grstavares/DadosAbertosTSE
Tem 5 Commits - 1 Contributors

>>>  https://api.github.com/repos/codigourbano/distritos-sp
Tem 2 Commits - 1 Contributors

>>>  https://api.github.com/repos/dfalbel/cota-parlamentar
Tem 23 Commits - 1 Contributors

>>>  https://api.github.com/repos/MarcosTrajano/ionicTabelaFipe
Tem 2 Commits - 1 Contributors

>>>  https://api.github.com/repos/gilsondev/awesome-projects
Tem 6 Commits - 2 Contributors

>>>  https://api.github.com/repos/mariohmol/voos-fab
Tem 17 Commits - 1 Contributors

>>>  https://api.github.com/repos/paraibatransparente/salarios-servidores
Tem 1 Commits - 1 Contributors

>>>  https://a

Tem 8 Commits - 1 Contributors

>>>  https://api.github.com/repos/georgemaia/dadosabertos
Tem 31 Commits - 1 Contributors

>>>  https://api.github.com/repos/petersonjr/dados_abertos
> Repositório tem size igual a 0

>>>  https://api.github.com/repos/GabrielLimaSnT/ProjetoDadosAbertosImpacta
Tem 74 Commits - 3 Contributors

>>>  https://api.github.com/repos/natalialionel/Dados-Abertos-Desafios-e-Oportunidades
Tem 1 Commits - 1 Contributors

>>>  https://api.github.com/repos/mhbsti/dados-abertos-curitiba
Tem 28 Commits - 1 Contributors

>>>  https://api.github.com/repos/transparencia-mg/blog-dados-abertos
Tem 4 Commits - 1 Contributors

>>>  https://api.github.com/repos/alexvlima/MTur-Dados-Turismo-por-UF
Tem 2 Commits - 1 Contributors

>>>  https://api.github.com/repos/thyall/Dados_UFRN
Tem 23 Commits - 3 Contributors

>>>  https://api.github.com/repos/dcarvalho/analise_dados_abertos
Tem 7 Commits - 2 Contributors

>>>  https://api.github.com/repos/paulo-neto/visualizacaoDadosAbertos
Te

Tem 47 Commits - 1 Contributors

>>>  https://api.github.com/repos/bozoh/meu-legislador
Tem 25 Commits - 1 Contributors

>>>  https://api.github.com/repos/marceloandriolli/dados_solarimetricos_brasil
Tem 3 Commits - 1 Contributors

>>>  https://api.github.com/repos/ggdrn/ClimMapView
Tem 10 Commits - 1 Contributors

>>>  https://api.github.com/repos/crislanio/Dados_abertos_PortalDaTransparencia
Tem 5 Commits - 1 Contributors

>>>  https://api.github.com/repos/mauriciovll/Dados-Abertos-SC-versao-TESTE-
Tem 1 Commits - 1 Contributors

>>>  https://api.github.com/repos/mateushtoledo/sysvis
Tem 3 Commits - 1 Contributors

>>>  https://api.github.com/repos/DeehSlash/EstruturaDeDadosHash
Tem 1 Commits - 1 Contributors

>>>  https://api.github.com/repos/danielCavalcanti553/angular-api-dados-iffar
Tem 5 Commits - 1 Contributors

>>>  https://api.github.com/repos/georgemaia/dadosabertos-mobilidade-rn-natal
Tem 18 Commits - 1 Contributors

>>>  https://api.github.com/repos/FabianoOLima/1_Dados-e-

Tem 5 Commits - 0 Contributors

>>>  https://api.github.com/repos/BrunoMoriB/stockmarket-manager
Tem 23 Commits - 1 Contributors

>>>  https://api.github.com/repos/hiperorganicos/openlab_datavis
Tem 5 Commits - 1 Contributors

>>>  https://api.github.com/repos/code-like-a-girl/hacker-cidadao-3.0
Tem 2 Commits - 1 Contributors

>>>  https://api.github.com/repos/NastyaCodingBunny/PYTHON---SIMULADOR-DE-CADASTRO
Tem 1 Commits - 1 Contributors

>>>  https://api.github.com/repos/wharborges/analiseES
> Repositório tem size igual a 0

>>>  https://api.github.com/repos/VRPazdeJesus/multas-ibama
Tem 2 Commits - 1 Contributors

>>>  https://api.github.com/repos/decosoares/diariasservidoresalagoas
Tem 18911 Commits - 315 Contributors

>>>  https://api.github.com/repos/leonardocouy/MaisCopa
Tem 9 Commits - 1 Contributors

>>>  https://api.github.com/repos/Idiego2/Dashboard-PHP-e-MySQL
Tem 6 Commits - 1 Contributors

>>>  https://api.github.com/repos/OLIVEIRAFAEL/GoogleFinance
Tem 3 Commits - 2 Cont

Tem 19 Commits - 1 Contributors

>>>  https://api.github.com/repos/leilton/PJIN
Tem 4 Commits - 1 Contributors

>>>  https://api.github.com/repos/desireesantos/plataformaVisualizacao
Tem 35 Commits - 1 Contributors

>>>  https://api.github.com/repos/Lorenaps/caracterizacao-ds-dados-abertos
Tem 25 Commits - 1 Contributors

>>>  https://api.github.com/repos/diegoep/rotasaude
Tem 1 Commits - 1 Contributors

>>>  https://api.github.com/repos/hitalosiqueira/DAG
Tem 1 Commits - 1 Contributors

>>>  https://api.github.com/repos/EnriqueSampaio/dag-parser
Tem 39 Commits - 1 Contributors

>>>  https://api.github.com/repos/edsonmottac/go-to-doc
Tem 5 Commits - 0 Contributors

>>>  https://api.github.com/repos/narallynne/mineracao-ogd
Tem 3 Commits - 2 Contributors

>>>  https://api.github.com/repos/mauriciovll/Dados-Abertos-SC-versao-TESTE-
Tem 1 Commits - 1 Contributors

>>>  https://api.github.com/repos/dadosgovbr/catalogos-dados-brasil
Tem 42 Commits - 4 Contributors

>>>  https://api.github.c

Tem 4 Commits - 1 Contributors

>>>  https://api.github.com/repos/Panizzo/transparencia
> Repositório tem size igual a 0

>>>  https://api.github.com/repos/marcotmotta/ibd-query-project
Tem 7 Commits - 1 Contributors

>>>  https://api.github.com/repos/antonivargas/desafio-camara
Tem 13 Commits - 1 Contributors

>>>  https://api.github.com/repos/ppKrauss/getcsv_stdOpenGov
> Repositório tem size igual a 0

>>>  https://api.github.com/repos/ces-jf/PSPA
Tem 34 Commits - 2 Contributors

>>>  https://api.github.com/repos/VitorRoque/Analise-Fila-o-Politica-BR
> Repositório tem size igual a 0

>>>  https://api.github.com/repos/danielmarcelino/rRAIS
> Repositório tem size igual a 0

>>>  https://api.github.com/repos/nosdeedson/Projeto-leitura-de-API-com-Docker
Tem 10 Commits - 1 Contributors

>>>  https://api.github.com/repos/chrmorais/scrapygdf
Tem 6 Commits - 1 Contributors

>>>  https://api.github.com/repos/SabrinyBezerra/Dashboard
Tem 3 Commits - 1 Contributors

>>>  https://api.github.com/

In [17]:
resultados.head(5)

,id,full_name,description,owner_type,owner_api_url,owner_url,url,api_url,fork,created_at,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,timestamp_extract,commits,contributors
0,29256552,CamaraDosDeputados/dados-abertos,Repositório do serviço de Dados Abertos da Câm...,Organization,https://api.github.com/users/CamaraDosDeputados,https://github.com/CamaraDosDeputados,https://github.com/CamaraDosDeputados/dados-ab...,https://api.github.com/repos/CamaraDosDeputado...,False,2015-01-14T17:32:49Z,...,None,True,True,7,7,203,135,1573174208,28,4
1,39256926,dadosgovbr/catalogos-dados-brasil,Mapeamento de iniciativas (e catálogos) de dad...,Organization,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/dadosgovbr/catalogos-dados-...,https://api.github.com/repos/dadosgovbr/catalo...,False,2015-07-17T14:02:34Z,...,Python,True,True,34,34,0,106,1573174208,42,4
2,73385196,prefeiturasp/dados-educacao,Análises e tutoriais das bases de dados aberto...,Organization,https://api.github.com/users/prefeiturasp,https://github.com/prefeiturasp,https://github.com/prefeiturasp/dados-educacao,https://api.github.com/repos/prefeiturasp/dado...,False,2016-11-10T13:35:40Z,...,Jupyter Notebook,True,True,18,18,1,48,1573174208,18,2
3,49720381,dadosgovbr/aplicativos-dados-brasil,Mapeamento de aplicativos e visualizações que ...,Organization,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/dadosgovbr/aplicativos-dado...,https://api.github.com/repos/dadosgovbr/aplica...,False,2016-01-15T13:29:14Z,...,None,True,True,9,9,1,48,1573174208,40,5
4,20032900,dadosgovbr/kit,Kit de implementação do Plano de Dados Abertos,Organization,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/dadosgovbr/kit,https://api.github.com/repos/dadosgovbr/kit,False,2014-05-21T18:44:02Z,...,CSS,True,False,15,15,10,39,1573174208,361,6


In [18]:
file_path = '../data/repositories_' + str(time.time()).split('.')[0] + '.csv'
file_path

'../data/repositories_1573177269.csv'

In [19]:
resultados.to_csv(file_path, index=False)

## Extraindo contribuidores dos repositórios

In [77]:
columns_contributors = [ 'repo_id',
                          'repo_name',
                          'repo_url',
                          'repo_api_url',
                          'contributor_id',
                          'contributor_login',
                          'contributor_type',
                          'contributor_url',
                          'contributor_api_url',
                          'timestamp_extract']

In [78]:
def scroll_results(results):
    
    list_contributors = []

    for result in results:
        contributor = {}
        
        contributor = {
            'contributor_id': result.get('id', None),
            'contributor_login': result.get('login', None),
            'contributor_type': result.get('type', None),
            'contributor_url': result.get('html_url', None),
            'contributor_api_url': result.get('url', None),
        }

        list_contributors.append(contributor)
        
    return list_contributors

In [83]:
def get_contributors(url):

    list_contributors = []
    results = requests.get(url, auth=credentials)
    
    # No caso do repositório estar vazio
    if results.status_code is 204:
        return None
    
    contributors_results = results.json()
    list_contributors = scroll_results(contributors_results)
    
    header = dict(results.links)
    
    while header.get('next', False):
        next_url = header.get('next').get('url')
        
        print('\t> Extraindo da próxima página: {0}'.format(next_url))
            
        results = requests.get(next_url, auth=credentials)
        contributors_results = results.json()
        list_contributors = list_contributors + scroll_results(contributors_results)    
        header = dict(results.links)
        
    return list_contributors

In [84]:
def add_contributor(repo, contributor):
            
    df = pd.DataFrame([[
                    repo.get('repo_id', None),
                    repo.get('repo_name', None),
                    repo.get('repo_url', None),      
                    repo.get('repo_api_url', None),
                    contributor.get('contributor_id', None),
                    contributor.get('contributor_login', None),
                    contributor.get('contributor_type', None),
                    contributor.get('contributor_url', None),
                    contributor.get('contributor_api_url', None),
                    str(time.time()).split('.')[0]]], columns=columns_contributors)

    return df  

In [85]:
def save_contributors(contributors_df, repo, contributors):
    
    for contributor in contributors:
                
        contributors_df = pd.concat([contributors_df, 
                                     add_contributor(repo, contributor)], 
                                    ignore_index=True, 
                                    sort=False)
    return contributors_df

In [88]:
def search_contributors(repositories_df):
    
    contributors_df = pd.DataFrame(columns = columns_contributors)
    urls = repositories_df['api_url']
   
    for url in urls[:5]:
        print('\nExtraindo contribuidores de: {0}'.format(url))

        repo = {
            'repo_id': repositories_df.loc[repositories_df["api_url"] == url, 'id'].values[0],
            'repo_name': repositories_df.loc[repositories_df["api_url"] == url, 'full_name'].values[0],
            'repo_url': repositories_df.loc[repositories_df["api_url"] == url, 'url'].values[0],
            'repo_api_url': url,
        }
        
        url_contributors = url + '/contributors'        
        contributors = get_contributors(url_contributors)
        
        if contributors:
            contributors_df = save_contributors(contributors_df, repo, contributors)
        
    
    return contributors_df

In [89]:
%%time
result_contributors = search_contributors(resultados)


>>> https://api.github.com/repos/CamaraDosDeputados/dados-abertos

>>> https://api.github.com/repos/dadosgovbr/catalogos-dados-brasil

>>> https://api.github.com/repos/prefeiturasp/dados-educacao
CPU times: user 113 ms, sys: 0 ns, total: 113 ms
Wall time: 2.23 s


In [92]:
result_contributors

,repo_id,repo_name,repo_url,repo_api_url,contributor_id,contributor_login,contributor_type,contributor_url,contributor_api_url,timestamp_extract
0,"0 29256552 Name: id, dtype: object",0 CamaraDosDeputados/dados-abertos Name: fu...,0 https://github.com/CamaraDosDeputados/dad...,https://api.github.com/repos/CamaraDosDeputado...,19875696,FabricioRocha,User,https://github.com/FabricioRocha,https://api.github.com/users/FabricioRocha,1573179625
1,"0 29256552 Name: id, dtype: object",0 CamaraDosDeputados/dados-abertos Name: fu...,0 https://github.com/CamaraDosDeputados/dad...,https://api.github.com/repos/CamaraDosDeputado...,16920325,EquipeDadosAbertosCD,User,https://github.com/EquipeDadosAbertosCD,https://api.github.com/users/EquipeDadosAbertosCD,1573179625
2,"0 29256552 Name: id, dtype: object",0 CamaraDosDeputados/dados-abertos Name: fu...,0 https://github.com/CamaraDosDeputados/dad...,https://api.github.com/repos/CamaraDosDeputado...,19963732,JoaoCarabetta,User,https://github.com/JoaoCarabetta,https://api.github.com/users/JoaoCarabetta,1573179625
3,"0 29256552 Name: id, dtype: object",0 CamaraDosDeputados/dados-abertos Name: fu...,0 https://github.com/CamaraDosDeputados/dad...,https://api.github.com/repos/CamaraDosDeputado...,7976552,labhacker,User,https://github.com/labhacker,https://api.github.com/users/labhacker,1573179625
4,1 39256926 370 39256926 427 3925692...,1 dadosgovbr/catalogos-dados-brasil 370 ...,1 https://github.com/dadosgovbr/catalogos...,https://api.github.com/repos/dadosgovbr/catalo...,1058414,augusto-herrmann,User,https://github.com/augusto-herrmann,https://api.github.com/users/augusto-herrmann,1573179626
5,1 39256926 370 39256926 427 3925692...,1 dadosgovbr/catalogos-dados-brasil 370 ...,1 https://github.com/dadosgovbr/catalogos...,https://api.github.com/repos/dadosgovbr/catalo...,42806423,filipecaldas,User,https://github.com/filipecaldas,https://api.github.com/users/filipecaldas,1573179626
6,1 39256926 370 39256926 427 3925692...,1 dadosgovbr/catalogos-dados-brasil 370 ...,1 https://github.com/dadosgovbr/catalogos...,https://api.github.com/repos/dadosgovbr/catalo...,13664229,rodrigogbrito,User,https://github.com/rodrigogbrito,https://api.github.com/users/rodrigogbrito,1573179626
7,1 39256926 370 39256926 427 3925692...,1 dadosgovbr/catalogos-dados-brasil 370 ...,1 https://github.com/dadosgovbr/catalogos...,https://api.github.com/repos/dadosgovbr/catalo...,17555132,jrsilva1984,User,https://github.com/jrsilva1984,https://api.github.com/users/jrsilva1984,1573179626
8,"2 73385196 Name: id, dtype: object",2 prefeiturasp/dados-educacao Name: full_na...,2 https://github.com/prefeiturasp/dados-edu...,https://api.github.com/repos/prefeiturasp/dado...,2335525,campagnucci,User,https://github.com/campagnucci,https://api.github.com/users/campagnucci,1573179627
9,"2 73385196 Name: id, dtype: object",2 prefeiturasp/dados-educacao Name: full_na...,2 https://github.com/prefeiturasp/dados-edu...,https://api.github.com/repos/prefeiturasp/dado...,28656406,tutss,User,https://github.com/tutss,https://api.github.com/users/tutss,1573179627


In [ ]:
file_path = '../data/contributors_' + str(time.time()).split('.')[0] + '.csv'
file_path

In [ ]:
result_contributors.to_csv(file_path, index=False)